## STRAICO

In [ ]:
import json
import csv
import requests
from typing import List, Optional, Dict, Any

# API endpoint
API_URL = "https://api.straico.com/v1/prompt/completion"

# Hardcoded API key and default model(s)
API_KEY = "QX-2ZLKkTvI1lr3MjeoHVD5LTz7iRsrHw09U80aHXA55bDURRmB"

PROMPT_ID_MAP = {
    "prompt_1": "P1", # Dada la palabra --> Definición del modismo.
    "prompt_2": "P2", # Dada la palabra --> Decir si es modismo o no.
    "prompt_3": "P3", # Dada la definición --> Dame la palabra / modismo.
    "prompt_4": "P4", # Dado el ejemplo --> Reemplazo literal y definición del remplazo.
}

### Call Models

#### Modelos Disponibles

In [2]:
def get_available_models() -> Dict[str, Any]:
    """Obtener la lista de modelos disponibles en Straico.
    
    Returns:
        Dict con la lista de modelos y su información (nombre, modelo, pricing, max_output)
    """
    models_url = "https://api.straico.com/v0/models"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    
    try:
        resp = requests.get(models_url, headers=headers, timeout=30)
    except requests.exceptions.RequestException as exc:
        return {"error": str(exc)}
    
    # Parse JSON
    try:
        data = resp.json()
    except Exception:
        return {"error": "No se pudo parsear la respuesta JSON"}
    
    if 200 <= resp.status_code < 300:
        return data
    
    return {"error": f"status={resp.status_code}", "response": data}


In [3]:
# Obtener la lista de modelos
models_data = get_available_models()

if "error" in models_data:
    print(f"Error: {models_data['error']}")
else:
    models_list = models_data.get('data', [])
    print(f"Total de modelos disponibles: {len(models_list)}\n")
    print("Lista de modelos:")
    print("-" * 80)
    for model in models_list:
        name = model.get('name', 'N/A')
        model_id = model.get('model', 'N/A')
        pricing = model.get('pricing', {})
        coins = pricing.get('coins', 'N/A')
        words = pricing.get('words', 'N/A')
        max_output = model.get('max_output', 'N/A')
        
        print(f"Nombre: {name}")
        print(f"  ID: {model_id}")
        print(f"  Precio: {coins} coins por {words} palabras")
        print(f"  Max Output: {max_output} tokens")
        print()

Total de modelos disponibles: 77

Lista de modelos:
--------------------------------------------------------------------------------
Nombre: Amazon: Nova Lite 1.0
  ID: amazon/nova-lite-v1
  Precio: 0.2 coins por 100 palabras
  Max Output: 5000 tokens

Nombre: Amazon: Nova Micro 1.0
  ID: amazon/nova-micro-v1
  Precio: 0.1 coins por 100 palabras
  Max Output: 5000 tokens

Nombre: Anthropic: Claude 3 Opus
  ID: anthropic/claude-3-opus
  Precio: 24 coins por 100 palabras
  Max Output: 4096 tokens

Nombre: Anthropic: Claude 3.5 Haiku
  ID: anthropic/claude-3-5-haiku-20241022
  Precio: 1.6 coins por 100 palabras
  Max Output: 8192 tokens

Nombre: Anthropic: Claude 3.5 Sonnet
  ID: anthropic/claude-3.5-sonnet
  Precio: 4.8 coins por 100 palabras
  Max Output: 8192 tokens

Nombre: Anthropic: Claude 3.7 Sonnet Reasoning (High)
  ID: anthropic/claude-3.7-sonnet:thinking
  Precio: 6 coins por 100 palabras
  Max Output: 8192 tokens

Nombre: Anthropic: Claude 3.7 Sonnet Reasoning (Medium)
  ID: a

#### Usar Modelos

In [4]:
def send_prompt(message: str, models: Optional[List[str]] = None) -> Dict[str, Any]:
    
    """ Send a single text prompt to Straico and return the parsed response.
        This function always performs a live HTTP request.
    """

    payload: Dict[str, Any] = {"models": models, "message": message}
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

    try:
        resp = requests.post(API_URL, headers=headers, json=payload, timeout=30)
    except requests.exceptions.RequestException as exc:
        return {"error": str(exc)}

    # Parse JSON if available
    try:
        data = resp.json()
    except Exception:
        data = None

    if 200 <= resp.status_code < 300:

        # Try to extract the assistant 'content' text from the common response
        # shape: data -> completions -> <model> -> completion -> choices[0] -> message -> content

        if isinstance(data, dict):
            completions = data.get('data', {}).get('completions', {})

            # If we passed a single model, try to use that key; otherwise take first
            model_key = None

            if models and len(models) == 1:
                model_key = models[0]
            if not model_key and isinstance(completions, dict) and len(completions) > 0:
                model_key = next(iter(completions.keys()))

            if model_key and model_key in completions:
                try:
                    content = completions[model_key]['completion']['choices'][0]['message']['content']
                    return content
                
                except Exception:
                    pass

        # Fallback: return raw text or the full JSON string
        if data is None:
            return resp.text
        
        try:
            return json.dumps(data, ensure_ascii=False)
        except Exception:
            return str(data)

    return {"error": f"status={resp.status_code}", "response": data if data is not None else resp.text}


### Configuración General

In [5]:
def _import_prompts() -> Dict[str, str]:
    """Load prompt_1..prompt_4 from prompts.py and return as a dict.

    If prompts.py isn't present, returns an empty dict.
    """
    try:
        from . import prompts as p  # type: ignore
    except Exception:
        try:
            import prompts as p  # type: ignore
        except Exception:
            return {}

    out: Dict[str, str] = {}
    for name in ("prompt_1", "prompt_2", "prompt_3", "prompt_4"):
        if hasattr(p, name):
            out[name] = getattr(p, name)
    return out


In [6]:
# Configuración
DEFAULT_MODELS = ["amazon/nova-micro-v1", "cohere/command-r-08-2024"]
DATASET_PATH = '../modismos_Dataset_Final.csv'
N_ROWS = 10

# Cargar prompts
PROMPTS = _import_prompts()
print(f"Prompts cargados: {list(PROMPTS.keys())}")

Prompts cargados: ['prompt_1', 'prompt_2', 'prompt_3', 'prompt_4']


In [7]:
def cargar_dataset(n_rows=None):
    """Carga el dataset y retorna una lista de diccionarios con modismo, significado y ejemplo."""
    rows = []
    seen_modismos = set()
    
    with open(DATASET_PATH, encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for r in reader:
            modismo = r.get('modismo', '').strip()
            if not modismo or modismo.casefold() in seen_modismos:
                continue
            seen_modismos.add(modismo.casefold())
            
            rows.append({
                'modismo': modismo,
                'significado': r.get('significado', '').strip(),
                'ejemplo': r.get('ejemplo', '').strip()
            })
            
            if n_rows and len(rows) >= n_rows:
                break
    
    return rows

def sanitize_column_name(model_name):
    """Convierte nombres de modelos en nombres válidos de columnas CSV."""
    return model_name.replace('/', '_').replace(':', '_').replace('-', '_')

def save_responses_to_csv(filepath, headers, data):
    """Guarda las respuestas en un archivo CSV con JSON compactado en una sola línea."""
    with open(filepath, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(headers)
        
        # Procesar cada fila para compactar JSON
        for row in data:
            processed_row = []
            for cell in row:
                # Si es un string que parece JSON, compactarlo
                if isinstance(cell, str) and (cell.strip().startswith('{') or cell.strip().startswith('[')):
                    try:
                        # Parsear y reserializar sin espacios ni saltos de línea
                        parsed = json.loads(cell)
                        cell = json.dumps(parsed, ensure_ascii=False, separators=(',', ':'))
                    except:
                        # Si no es JSON válido, dejar como está
                        pass
                processed_row.append(cell)
            writer.writerow(processed_row)
    
    print(f"Guardado: {filepath}")

---
## PROMPT 1: Modismo → Definición

**Input**: `modismo`  
**Output**: `definicion`  
**CSV**: `responses_prompt_1.csv`

In [8]:
def run_prompt_1(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 1: Dada la palabra/modismo → Generar definición
    INPUT: modismo
    OUTPUT: definicion
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 1: Modismo -> Definicion")
    print("=" * 80)
    
    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    print(f"Dataset cargado: {len(dataset)} filas")
    
    # Obtener template del prompt
    template = PROMPTS.get('prompt_1')
    if not template:
        print("ERROR: prompt_1 no encontrado")
        return
    
    # Preparar CSV
    output_file = 'responses_prompt_1.csv'
    headers = ['modismo'] + [sanitize_column_name(m) for m in models]
    results = []
    
    # Procesar cada fila
    for idx, row in enumerate(dataset, 1):
        modismo = row['modismo']
        print(f"\n[{idx}/{len(dataset)}] Procesando: {modismo}")
        
        # Armar el prompt reemplazando placeholders
        prompt_text = template.replace('{{modismo}}', modismo)
        
        # Obtener respuestas de cada modelo
        responses = []
        for model in models:
            print(f"  -> Consultando {model}...", end=" ")
            resp = send_prompt(prompt_text, models=[model])
            # Compactar JSON en una sola línea
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    resp_str = json.dumps(parsed, ensure_ascii=False, separators=(',', ':'))
                except:
                    resp_str = resp
            elif isinstance(resp, dict):
                if 'error' in resp:
                    resp_str = json.dumps(resp, ensure_ascii=False, separators=(',', ':'))
                else:
                    resp_str = str(resp)
            else:
                resp_str = str(resp)
            responses.append(resp_str)
            print("")
        
        results.append([modismo] + responses)
    
    # Guardar CSV
    print()
    save_responses_to_csv(output_file, headers, results)

In [9]:
# Ejecutar Prompt 1
run_prompt_1()

EJECUTANDO PROMPT 1: Modismo -> Definicion
Dataset cargado: 10 filas

[1/10] Procesando: abajeño
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[2/10] Procesando: abalear
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[3/10] Procesando: abaleo
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[4/10] Procesando: abanicar
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[5/10] Procesando: abanico
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[6/10] Procesando: abarco
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[7/10] Procesando: abarrotes
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[8/10] Procesando: abeja
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[9/

---
## PROMPT 2: Modismo → Es Modismo (Sí/No)

**Input**: `modismo`  
**Output**: `es_modismo` (Sí/No)  
**CSV**: `responses_prompt_2.csv`

In [10]:
def run_prompt_2(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 2: Dada la palabra/modismo → Determinar si es modismo (Sí/No)
    INPUT: modismo
    OUTPUT: es_modismo
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 2: Modismo -> Es Modismo (Si/No)")
    print("=" * 80)
    
    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    print(f"Dataset cargado: {len(dataset)} filas")
    
    # Obtener template del prompt
    template = PROMPTS.get('prompt_2')
    if not template:
        print("ERROR: prompt_2 no encontrado")
        return
    
    # Preparar CSV
    output_file = 'responses_prompt_2.csv'
    headers = ['modismo'] + [sanitize_column_name(m) for m in models]
    results = []
    
    # Procesar cada fila
    for idx, row in enumerate(dataset, 1):
        modismo = row['modismo']
        print(f"\n[{idx}/{len(dataset)}] Procesando: {modismo}")
        
        # Armar el prompt reemplazando placeholders
        prompt_text = template.replace('{{modismo}}', modismo)
        
        # Obtener respuestas de cada modelo
        responses = []
        for model in models:
            print(f"  -> Consultando {model}...", end=" ")
            resp = send_prompt(prompt_text, models=[model])
            # Compactar JSON en una sola línea
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    resp_str = json.dumps(parsed, ensure_ascii=False, separators=(',', ':'))
                except:
                    resp_str = resp
            elif isinstance(resp, dict):
                if 'error' in resp:
                    resp_str = json.dumps(resp, ensure_ascii=False, separators=(',', ':'))
                else:
                    resp_str = str(resp)
            else:
                resp_str = str(resp)
            responses.append(resp_str)
            print("")
        
        results.append([modismo] + responses)
    
    # Guardar CSV
    print()
    save_responses_to_csv(output_file, headers, results)

In [11]:
# Ejecutar Prompt 2
run_prompt_2()

EJECUTANDO PROMPT 2: Modismo -> Es Modismo (Si/No)
Dataset cargado: 10 filas

[1/10] Procesando: abajeño
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[2/10] Procesando: abalear
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[3/10] Procesando: abaleo
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[4/10] Procesando: abanicar
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[5/10] Procesando: abanico
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[6/10] Procesando: abarco
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[7/10] Procesando: abarrotes
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[8/10] Procesando: abeja
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024.

---
## PROMPT 3: Definición → Modismo

**Input**: `significado/definicion`  
**Output**: `palabra` (modismo identificado)  
**CSV**: `responses_prompt_3.csv`

In [12]:
def run_prompt_3(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 3: Dada la definición → Generar modismo
    INPUT: definicion (significado del dataset)
    OUTPUT: palabra (modismo)
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 3: Definicion -> Modismo")
    print("=" * 80)
    
    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    print(f"Dataset cargado: {len(dataset)} filas")
    
    # Obtener template del prompt
    template = PROMPTS.get('prompt_3')
    if not template:
        print("ERROR: prompt_3 no encontrado")
        return
    
    # Preparar CSV
    output_file = 'responses_prompt_3.csv'
    headers = ['significado', 'modismo_original'] + [sanitize_column_name(m) for m in models]
    results = []
    
    # Procesar cada fila
    for idx, row in enumerate(dataset, 1):
        significado = row['significado']
        modismo_original = row['modismo']
        
        # Saltar si no hay significado
        if not significado:
            print(f"\n[{idx}/{len(dataset)}] ADVERTENCIA: Saltando (sin significado): {modismo_original}")
            continue
            
        print(f"\n[{idx}/{len(dataset)}] Procesando definicion de: {modismo_original}")
        print(f"  Definicion: {significado[:60]}...")
        
        # Armar el prompt reemplazando placeholders
        prompt_text = template.replace('{{definicion}}', significado)
        
        # Obtener respuestas de cada modelo
        responses = []
        for model in models:
            print(f"  -> Consultando {model}...", end=" ")
            resp = send_prompt(prompt_text, models=[model])
            # Compactar JSON en una sola línea
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    resp_str = json.dumps(parsed, ensure_ascii=False, separators=(',', ':'))
                except:
                    resp_str = resp
            elif isinstance(resp, dict):
                if 'error' in resp:
                    resp_str = json.dumps(resp, ensure_ascii=False, separators=(',', ':'))
                else:
                    resp_str = str(resp)
            else:
                resp_str = str(resp)
            responses.append(resp_str)
            print("")
        
        results.append([significado, modismo_original] + responses)
    
    # Guardar CSV
    print()
    save_responses_to_csv(output_file, headers, results)

In [13]:
# Ejecutar Prompt 3
run_prompt_3()

EJECUTANDO PROMPT 3: Definicion -> Modismo
Dataset cargado: 10 filas

[1/10] Procesando definicion de: abajeño
  Definicion: Propio o nativo de las costas o de las tierras bajas....
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[2/10] Procesando definicion de: abalear
  Definicion: Disparar a alguien o a algo de manera repetida con un arma d...
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[3/10] Procesando definicion de: abaleo
  Definicion: Situación en la que hay disparos repetidos que provienen de ...
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[4/10] Procesando definicion de: abanicar
  Definicion: En el beisbol, fallar el bateador al no tocar la pelota que ...
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[5/10] Procesando definicion de: abanico
  Definicion: Aparato eléctrico que crea una corriente de aire e

---
## PROMPT 4: Modismo + Ejemplo → Literal + Definición

**Input**: `modismo` + `ejemplo`  
**Output**: `literal` + `definicion`  
**CSV**: `responses_prompt_4.csv`

In [14]:
def run_prompt_4(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 4: Dado modismo + ejemplo → Generar literal + definición
    INPUT: modismo + ejemplo
    OUTPUT: literal + definicion
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 4: Modismo + Ejemplo -> Literal + Definicion")
    print("=" * 80)
    
    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    print(f"Dataset cargado: {len(dataset)} filas")
    
    # Obtener template del prompt
    template = PROMPTS.get('prompt_4')
    if not template:
        print("ERROR: prompt_4 no encontrado")
        return
    
    # Preparar CSV
    output_file = 'responses_prompt_4.csv'
    headers = ['modismo', 'ejemplo'] + [sanitize_column_name(m) for m in models]
    results = []
    
    # Procesar cada fila
    for idx, row in enumerate(dataset, 1):
        modismo = row['modismo']
        ejemplo = row['ejemplo']
        
        # Saltar si no hay ejemplo
        if not ejemplo:
            print(f"\n[{idx}/{len(dataset)}] ADVERTENCIA: Saltando (sin ejemplo): {modismo}")
            continue
        
        print(f"\n[{idx}/{len(dataset)}] Procesando: {modismo}")
        print(f"  Ejemplo: {ejemplo[:60]}...")
        
        # Armar el prompt reemplazando placeholders
        prompt_text = template.replace('{{modismo}}', modismo).replace('{{ejemplo}}', ejemplo)
        
        # Obtener respuestas de cada modelo
        responses = []
        for model in models:
            print(f"  -> Consultando {model}...", end=" ")
            resp = send_prompt(prompt_text, models=[model])
            # Compactar JSON en una sola línea
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    resp_str = json.dumps(parsed, ensure_ascii=False, separators=(',', ':'))
                except:
                    resp_str = resp
            elif isinstance(resp, dict):
                if 'error' in resp:
                    resp_str = json.dumps(resp, ensure_ascii=False, separators=(',', ':'))
                else:
                    resp_str = str(resp)
            else:
                resp_str = str(resp)
            responses.append(resp_str)
            print("")
        
        results.append([modismo, ejemplo] + responses)
    
    # Guardar CSV
    print()
    save_responses_to_csv(output_file, headers, results)

In [15]:
# Ejecutar Prompt 4
run_prompt_4()

EJECUTANDO PROMPT 4: Modismo + Ejemplo -> Literal + Definicion
Dataset cargado: 10 filas

[1/10] Procesando: abajeño
  Ejemplo: Empezaron cultivos de tabaco y a elaborar un delicioso aguar...
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[2/10] Procesando: abalear
  Ejemplo: El funcionario resultó ileso a pesar de que el avión en que ...
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[3/10] Procesando: abaleo
  Ejemplo: Por la calle doce se oía un tremendo abaleo y al poco rato v...
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[4/10] Procesando: abanicar
  Ejemplo: El bateador se ponchó abanicando el tercer lanzamiento....
  -> Consultando amazon/nova-micro-v1... 
  -> Consultando cohere/command-r-08-2024... 

[5/10] Procesando: abanico
  Ejemplo: No podía dormir la siesta de la tarde sin el abanico prendid...
  -> Consultando amazon/nova-micro-v1... 
  -> Co